# This notebook contains your standard data exploration

In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import re
from datetime import datetime
from sklearn.linear_model import LinearRegression
import numpy as np
import seaborn as sns

pd.set_option('display.max_columns', None)


In [ ]:
price_data_dir = '../scrapers/asuntojen-hintatiedot/scraped_data/'
assert os.path.exists(price_data_dir), f'directory {price_data_dir} did not exist!'

for path, dirs, files in os.walk(price_data_dir):
    break
    
print(files)

In [ ]:
file_path = price_data_dir + '2021-04-19_Helsinki.csv'
data = pd.read_csv(file_path)

In [ ]:
data

## Data wrangling happens here

In [ ]:
def get_number_of_rooms(data):
    room_arrangement = data["room_arrangement"]
    if not type(room_arrangement) == str:
        return 0
    if (room_arrangement == 'AH'):
        # AH = asuinhuoneisto = single room, no toilet/kitchen/etc, use special value
        return 0.5
    variable_rooms_regexp = re.compile('([0-9])-([0-9])\s?h', re.IGNORECASE)
    variable_rooms = variable_rooms_regexp.findall(room_arrangement)
    if len(variable_rooms) > 0:
        return ((int(variable_rooms[0][0]) + int(variable_rooms[0][1])) / 2)
    simple_rooms_regexp = re.compile('([0-9])\s?h', re.IGNORECASE)
    simple_rooms = simple_rooms_regexp.findall(room_arrangement)
    if len(simple_rooms) > 0:
        return int(simple_rooms[0])
    return 0

def correct_neighbourhood(data):
    if type(data["neighbourhood"]) == str:
        if 'tapiola' in data["neighbourhood"].lower():
            return 'Tapiola'
        neighborhood_regexp = re.compile('^[a-zöä\-]*', re.IGNORECASE)
        neighborhood = neighborhood_regexp.findall(data["neighbourhood"])
        if len(neighborhood) > 0:
            return neighborhood[0]
    return ''

def calculate_neighbourhood_avg_price(data):
    neighborhood_prices = {}
    grouped = data.groupby('neighbourhood')
    for name, group in grouped:
        neighborhood_prices[name] = round(sum(group["price_per_square_meters"]) / len(group["price_per_square_meters"]))
    return neighborhood_prices

def set_neighbourhood_price(prices):
    def set_price(data):
        return prices[data["neighbourhood"]]
    return set_price


In [ ]:
data["number_of_rooms"] = data.apply(get_number_of_rooms, axis=1)
data["shape_is_good"] = (data["shape"] == 'good') * 1
data["has_elevator"] = data["has_elevator"] * 1
data["age"] = datetime.today().year - data["built_in"]
data["is_apartment"] = (data["house_type"] == "apartment") * 1
data["is_rowhouse"] = (data["house_type"] == "rowhouse") * 1
data["is_townhouse"] = (data["house_type"] == "townhouse") * 1
data["lot_is_owned"] = (data["lot"] == "owned") * 1

# neighbourhoods
data['neighbourhood'] = data['neighborhood'].replace(np.nan, '')
data["neighbourhood"] = data.apply(correct_neighbourhood, axis=1)
data['neighbourhood'] = data['neighbourhood'].replace('neighbourhood', 'unknown')

neighbourhood_prices = calculate_neighbourhood_avg_price(data)
data["neighbourhood_avg_price"] = data.apply(set_neighbourhood_price(neighbourhood_prices), axis=1)
data

In [ ]:
data.sort_values('neighborhood').neighborhood.unique()

In [ ]:
neighbourhood_prices

In [ ]:
neigh_rooms_df = data.groupby(['neighbourhood', 'house_type', 'number_of_rooms']).mean().reset_index()[['neighbourhood', 'house_type', 'number_of_rooms', 'square_meters', 'price_including_loans', 'price_per_square_meters', 'age', 'neighbourhood_avg_price']]
neigh_rooms_df['diff_neigh_avg'] = neigh_rooms_df['price_per_square_meters'] - neigh_rooms_df['neighbourhood_avg_price']
neigh_rooms_df

In [ ]:
neighbourhood_prices_df = pd.DataFrame(index=['price_m2'], data=neighbourhood_prices).T
neighbourhood_wl = ['Oulunkylä', 'Patola',
                             'Munkkiniemi',
                             'Munkkivuori',
                             #'Pikku huopalahti', 
                             'Pikku-huopalahti', 
                             'Pohjois-haaga', 
                             #'Pohjois haaga', 
                             'Etelä-haaga', 
                             'Lassila', 'Meilahti', 
                             #'Pikkuhuopalahti'
                            ]
neighbourhood_prices_df.loc[neighbourhood_wl]

## Visualize data

In [ ]:
def show_values_on_bars(axs, perc=False, round_to=0):
    def _show_on_single_plot(ax):
        for p in ax.patches:
            if np.isnan(p.get_height()):
                continue
            _x = p.get_x() + p.get_width() / 2
            _y = p.get_y() + p.get_height()
            _h = p.get_height()
            value = _h * 100 if perc else _h
            value = round(value, round_to)
            value = str(int(value)) if round_to == 0 else str(value)
            ax.text(_x, _y, value, ha="center", fontsize=8)

    if isinstance(axs, np.ndarray):
        for idx, ax in np.ndenumerate(axs):
            _show_on_single_plot(ax)
    else:
        _show_on_single_plot(axs)


#### Neighbourhoods 

In [ ]:
fig, ax = plt.subplots(figsize=(24, 10), dpi=140, facecolor='w')

colors = plt.get_cmap('plasma')(np.arange(0, data.shape[0]))
neighbourhood_data = data[["neighbourhood", "neighbourhood_avg_price"]].drop_duplicates().sort_values(by='neighbourhood_avg_price')

ax.bar(neighbourhood_data["neighbourhood"], neighbourhood_data["neighbourhood_avg_price"], color=colors)
plt.yticks(list(range(0,15000, 1000)))
plt.xticks(rotation=90, fontsize=7)
plt.title("Neigborhood price range")
plt.ylabel("Price, €")
plt.xlabel("Neighborhood")
plt.margins(0)
plt.grid()

plt.show()

#### Neighbourhoods by house type and number of rooms 

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8), dpi=140, facecolor='w')

htype = 'apartment'
neighbourhoods_wl = [
    'Oulunkylä', 
    'Patola',        
    'Meilahti', 
    'Munkkiniemi',
    'Munkkivuori',
    'Pikkuhuopalahti',
    'Pikku huopalahti', 
    'Pikku-huopalahti', 
    'Pohjois-haaga', 
    'Pohjois haaga', 
    'Etelä-haaga', 
    'Lassila',
    'Kannelmäki',
]

mask = neigh_rooms_df.neighbourhood.apply(lambda n: n in neighbourhoods_wl)
to_plot_df = neigh_rooms_df[mask]
to_plot_df = to_plot_df.query(f'house_type == "{htype}"')

ax = sns.barplot(data=to_plot_df,
                x='neighbourhood',
                y='price_per_square_meters', 
                hue='number_of_rooms'
           )

show_values_on_bars(ax)
ax.legend(bbox_to_anchor=(1.01, 1))

ax.set_title(f'average price of {htype}s per m2 in chosen neighborhoods by number of rooms')

plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
house_type_colors = {
    'is_apartment': 'purple',
    'is_rowhouse': 'cyan',
    'is_townhouse': 'blue'
}
for house_type in ["is_apartment", "is_rowhouse", "is_townhouse"]:
    type_data = data[data[house_type] == 1]
    ax.scatter(type_data["square_meters"], type_data["price_including_loans"], color=house_type_colors[house_type], label=house_type)
    
plt.title("Apartment size in square meters vs price")
plt.ylabel("Price, m€")
plt.xlabel("Size, m^2")
plt.legend()

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))

ax.scatter(data["square_meters"], data["number_of_rooms"])
    
plt.title("Apartment size square meters vs room numbers")
plt.ylabel("Number of rooms")
plt.xlabel("Size, m^2")

plt.show()

### Thoughts on apartment size vs room numbers
- quite a lot of 0s in there, might need some extra cleaning on the wrangling part?
- looks like there is a pretty linear dependency between number of rooms and size, which is again unsuprising


In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
house_type_colors = {
    'is_apartment': 'purple',
    'is_rowhouse': 'cyan',
    'is_townhouse': 'blue'
}
for house_type in ["is_apartment", "is_rowhouse", "is_townhouse"]:
    type_data = data[data[house_type] == 1]
    ax.scatter(type_data["age"], type_data[ "price_including_loans"], color=house_type_colors[house_type], label=house_type)
    
plt.title("House age vs price")
plt.ylabel("Price, m€")
plt.xlabel("Age, years")
plt.legend()

plt.show()

### Thoughts on age vs price
- well, rather surprisingly, apartment age does not seem to correlate with its price

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
house_type_colors = {
    'is_apartment': 'purple',
    'is_rowhouse': 'cyan',
    'is_townhouse': 'blue'
}

for house_type in ["is_apartment", "is_rowhouse", "is_townhouse"]:
    type_data = data[data[house_type] == 1]
    scale = 25 + (75 * type_data["shape_is_good"])
    ax.scatter(type_data["age"], type_data[ "price_including_loans"], color=house_type_colors[house_type], label=house_type, s=scale)
    
plt.title("House age vs price")
plt.ylabel("Price, m€")
plt.xlabel("Age, years")
plt.legend()

plt.show()

In [ ]:
f'C{data["lot_is_owned"].iloc[0]}'

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))

for house_type in [1, 0]:
    type_data = data[data["lot_is_owned"] == house_type]
    ax.scatter(type_data["square_meters"], type_data[ "price_including_loans"], c=f'C{type_data["lot_is_owned"].iloc[0]}', label=house_type)
    
plt.title("Apartment size vs price by lot owned(1) or rented(0)")
plt.ylabel("Price, m€")
plt.xlabel("Size in square meters")
plt.legend()

plt.show()

## Do regression!

In [ ]:
data.columns

In [ ]:
x = data[['square_meters', 'has_elevator', 'shape_is_good', 'age', 'is_apartment','is_rowhouse', 'is_townhouse', 'lot_is_owned']]
y = data["price_including_loans"]


In [ ]:

model = LinearRegression().fit(x, y)

In [ ]:
model.coef_

In [ ]:
candidate_apartment = np.array([50, 1, 1, 30, 0, 1, 0, 0]).reshape(1, -1)
model.predict(candidate_apartment)

## Only apartments!

In [ ]:
htype = 'apartment'
neighbourhoods_wl = [
    'Oulunkylä', 
    'Patola',        
    #'Meilahti', 
    #'Munkkiniemi',
    'Munkkivuori',
    #'Pikkuhuopalahti',
    #'Pikku huopalahti', 
    #'Pikku-huopalahti', 
    'Pohjois-haaga', 
    'Pohjois haaga', 
    #'Etelä-haaga', 
    'Lassila',
    'Kannelmäki',
]

mask = data.neighbourhood.apply(lambda n: n in neighbourhoods_wl)
train_df = data[mask]
train_df = train_df.query(f'house_type == "{htype}"')

train_df

In [ ]:
x = train_df[['square_meters', 'shape_is_good', 'age', 'has_elevator', 'lot_is_owned']]
y = train_df["price_including_loans"]

In [ ]:
model = LinearRegression().fit(x, y)

In [ ]:
model.coef_

In [ ]:
candidate = np.array([44.6, 1, 50, 1, 0]).reshape(1, -1)
cand_ren = 45000
cand_price = 189000
model.predict(candidate)

## Plot candidate

In [ ]:
sq_x = np.array(train_df['square_meters']).reshape(-1, 1)
sq_y = train_df["price_including_loans"]
sm_model = LinearRegression().fit(sq_x, sq_y)

In [ ]:
sm_coef = sm_model.coef_
sm_intercept = sm_model.intercept_

In [ ]:

pred, asked, asked_w_ren = model.predict(candidate), cand_price, cand_price + cand_ren

fig, ax = plt.subplots(figsize=(12, 8), facecolor='w', dpi=150)

ax.scatter(train_df["square_meters"], train_df[ "price_including_loans"])

ax.scatter(44.6, pred, label=f"Candidate predicted price {pred[0]//1000}k€", s=100)
ax.scatter(44.6, asked, label=f"Candidate asked price {asked//1000}k€", s=100)
ax.scatter(44.6, asked_w_ren, label=f"Candidate price w estimated 10y renovations {asked_w_ren//1000}k€", s=100)
sm_max = 120
plt.plot((0, sm_max), (sm_intercept, sm_max*sm_coef + sm_intercept), c='m', label="Chosen neighbourhoods estimated price, aka market price")

plt.title(f"m2 vs price in chosen neighbourhoods\n{neighbourhoods_wl}")
plt.ylabel("Price (€)")
plt.xlabel("Size in square meters")
plt.grid(True)
plt.legend()

plt.tight_layout()

plt.show()

#### Estimate value gain 

In [ ]:
cand_price, cand_ren

In [ ]:
offer = -9000

In [ ]:
selling_price = cand_price + offer
selling_price

In [ ]:
money_in = selling_price + cand_ren
money_in

In [ ]:
expected_revenue_perc = 7

In [ ]:
required_reselling_price = money_in * (100 + expected_revenue_perc) / 100
required_reselling_price

In [ ]:
reselling_price = 240000
reselling_price

In [ ]:
profit = reselling_price - money_in
profit

In [ ]:
profit_perc = profit / money_in * 100 // 1
profit_perc

#### Helsinki € / m2 development for past 2y 

In [ ]:
two_y_ago = 5422
last_mon = 6130

incr_perc = (last_mon - two_y_ago) / two_y_ago * 100
incr_perc

#### Two room apartments in Helsinki 2y dev 

In [ ]:
two_y_ago = 5334
last_mon = 5881
incr_perc = (last_mon - two_y_ago) / two_y_ago * 100
incr_perc